# Gold Layer 테스트 노트북

dbt로 생성된 Gold 테이블 확인용

In [ ]:
import sys
sys.path.append('/app')

from src.utils.spark_builder import get_spark_session
from pyspark.sql import functions as F

# Spark 세션 생성
spark = get_spark_session("GDELT_Gold_Test", "spark://spark-master:7077")
print("✅ Spark 세션 생성 완료")

In [ ]:
# Gold Table 읽기 (dbt가 생성한 테이블)
gold_df = spark.read.format("delta").load("s3a://gold/gdelt_microbatch_country_analysis")

print(f"📊 Gold 테이블 레코드 수: {gold_df.count()}")
print(f"📋 컬럼 수: {len(gold_df.columns)}")

In [ ]:
# Gold 테이블 스키마 확인
print("📝 Gold 테이블 컬럼:")
for i, col in enumerate(gold_df.columns, 1):
    print(f"{i:2d}. {col}")

print("\n🔍 스키마:")
gold_df.printSchema()

In [ ]:
# Gold 데이터 샘플 확인
print("🔍 Gold 테이블 샘플 데이터:")
gold_df.show(10, truncate=False)

In [ ]:
# 국가별 이벤트 통계
print("📊 국가별 총 이벤트 수 (Top 10):")
gold_df.groupBy("actor1_country_code") \
    .agg(F.sum("total_events").alias("total_country_events")) \
    .orderBy(F.desc("total_country_events")) \
    .show(10)

In [ ]:
# 이벤트 타입별 통계
print("📊 이벤트 타입별 총 이벤트 수 (Top 10):")
gold_df.groupBy("event_root_code") \
    .agg(F.sum("total_events").alias("total_event_type")) \
    .orderBy(F.desc("total_event_type")) \
    .show(10)

In [ ]:
# 톤 분석
print("📊 전체 톤 분석:")
gold_df.select(
    F.avg("average_tone").alias("overall_avg_tone"),
    F.min("min_tone").alias("global_min_tone"),
    F.max("max_tone").alias("global_max_tone")
).show()

In [ ]:
spark.stop()
print("✅ 테스트 완료!")